In [12]:
import cv2 as cv
import tifffile as tif
import xml.etree.ElementTree as ET
import numpy as np
import os
import glob
import tqdm
from tqdm import tqdm

In [30]:
# Replace with your OME-TIFF file path
filename = '/Volumes/T9/Virtual_Histology/source_data/HE_images/SR007-CL2-11c13c-188um-HE-20231018-s1.ome.tiff'

with tif.TiffFile(filename) as tif:
    ome_xml = tif.ome_metadata

# Parse the XML to find pixel size info
root = ET.fromstring(ome_xml)
namespace = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}

# Find the first Pixels element (adjust if needed for multi-dimensional data)
pixels = root.find('.//ome:Pixels', namespace)
if pixels is not None:
    pixel_size_x = pixels.get('PhysicalSizeX')
    pixel_size_y = pixels.get('PhysicalSizeY')
    unit = pixels.get('PhysicalSizeXUnit')  # typically 'µm'
    print(f"Pixel Size X: {pixel_size_x} {unit}")
    print(f"Pixel Size Y: {pixel_size_y} {unit}")
else:
    print("No pixel size metadata found.")


Pixel Size X: 0.172168225 None
Pixel Size Y: 0.172168225 None


Prepare domain Y dataset by downscaling

In [ ]:
import cv2 as cv
import os
import glob
from tqdm import tqdm  # Import tqdm for progress bar

def downscale_images(source_folder, output_folder, scale_factor=4/20):
    """
    Downscale all .tiff images in the source folder and save to the output folder.
    
    Parameters:
    - source_folder (str): Path to the HE_images folder.
    - output_folder (str): Where to save downscaled images.
    - scale_factor (float): Downscaling factor (default: 4/40).
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Recursively find all TIFF images in the folder
    tiff_files = glob.glob(os.path.join(source_folder, "**", "*.tiff"), recursive=True)

    if not tiff_files:
        print("No TIFF images found in the source directory.")
        return
    print(f"Found {len(tiff_files)} images. Processing...\n")

    # Initialize tqdm progress bar
    for file in tqdm(tiff_files, desc="Processing Images", unit="image"):
        try:
            # Load the image
            image = cv.imread(file)
            
            if image is None:
                print(f"Skipping: Unable to load {file}")
                continue

            # Downscale the image
            downscaled_image = cv.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv.INTER_AREA)

            # Define output path
            filename = os.path.basename(file)  # Keep original filename
            save_path = os.path.join(output_folder, filename)
 
            # Save downscaled image
            cv.imwrite(save_path, downscaled_image)

        except Exception as e:
            print(f"Error processing {file}: {e}")
    print("\nProcessing complete!")

source_folder = "/Volumes/T9/Virtual_Histology/source_data/HE_images"
output_folder = "/Volumes/T9/Virtual_Histology/primary_data/HE_images"  
downscale_images(source_folder, output_folder)


In [38]:
import os
import numpy as np
import tifffile as tif
from tqdm import tqdm

def crop_and_tile_tif_grid(image_path, crop_size=(1536, 1536), grid=(3, 3), output_dir="tiles"):
    # Load the full-resolution image using series=0
    img = tif.imread(image_path, series=0)

    height, width = img.shape[:2]
    desired_crop_width, desired_crop_height = crop_size
    grid_rows, grid_cols = grid

    # Adjust crop size so it's exactly divisible by the grid dimensions
    effective_crop_width = (desired_crop_width // grid_cols) * grid_cols
    effective_crop_height = (desired_crop_height // grid_rows) * grid_rows

    # Center the effective crop on the image
    left = (width - effective_crop_width) // 2
    top = (height - effective_crop_height) // 2
    cropped_img = img[top:top+effective_crop_height, left:left+effective_crop_width]

    # Determine tile dimensions
    tile_w = effective_crop_width // grid_cols
    tile_h = effective_crop_height // grid_rows

    # Get the base image name (without extension) to include in tile filenames
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    counter = 0
    # Loop over the grid to extract non-overlapping tiles
    for i in range(grid_rows):
        for j in range(grid_cols):
            row_start = i * tile_h
            row_end = (i + 1) * tile_h
            col_start = j * tile_w
            col_end = (j + 1) * tile_w
            tile = cropped_img[row_start:row_end, col_start:col_end]
            tile_filename = f"{image_name}_tile_{counter:03}.tiff"
            tile_path = os.path.join(output_dir, tile_filename)
            tif.imwrite(tile_path, tile)
            counter += 1

def process_directory_grid(input_dir, output_dir, crop_size=(1536, 1536), grid=(3, 3)):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # List only files ending with "ome.tiff" in the input directory
    files = [f for f in os.listdir(input_dir) if f.lower().endswith("ome.tiff")]
    
    # Process each OME-TIFF file with a tqdm progress bar
    for filename in tqdm(files, desc="Processing OME-TIFF images"):
        image_path = os.path.join(input_dir, filename)
        try:
            crop_and_tile_tif_grid(image_path, crop_size=crop_size, grid=grid, output_dir=output_dir)
        except tif.TiffFileError as e:
            pass
# Specify your input and output directories
input_dir = "/Volumes/T9/Virtual_Histology/primary_data/HE_images_downscaled"
output_dir = "/Volumes/T9/Virtual_Histology/primary_data/HE_images_patched"
crop_size = (1536, 1536)
grid = (3, 3)

process_directory_grid(input_dir, output_dir, crop_size=crop_size, grid=grid)


Processing OME-TIFF images: 100%|██████████| 186/186 [00:25<00:00,  7.19it/s]


Prepare domain X dataset

In [2]:
import os
import numpy as np
import tifffile as tif
import dask.array as da
from dask import delayed, compute
from tqdm import tqdm
import dask

# Import skimage functions for filtering and conversion.
from skimage.filters import unsharp_mask
from skimage.morphology import white_tophat, black_tophat, disk
from skimage.util import img_as_float, img_as_uint

def get_image_from_zarr_as_dask_array(zarr_path):
    # Load the Zarr file as a Dask array.
    zimg = da.from_zarr(zarr_path, component="muse/stitched/")
    return zimg

def apply_filters(img):
    """
    Apply an unsharp mask filter and morphological white top-hat and black-hat filters.
    The image is converted to float in [0,1] for filtering, then back to uint16.
    This process does not change the resolution.
    """
    # Convert to float
    img_float = img_as_float(img)
    # Apply unsharp mask (adjust radius and amount as needed)
    sharp = unsharp_mask(img_float, radius=1, amount=3)
    return sharp

@delayed
def process_tile(tile, scale_12bit):
    # Process a tile with optional 12-bit scaling.
    tile = tile.compute(scheduler='synchronous') if hasattr(tile, 'compute') else tile
    if tile.size == 0:
        return None
    if scale_12bit:
        tile = tile.astype(np.float32)
        tile_max = tile.max()
        scale_factor = 16 if (tile_max > 0 and abs(tile_max - 4095) / 4095 < 0.1) else (65535 / tile_max if tile_max > 0 else 1)
        tile = (tile * scale_factor).clip(0, 65535).astype(np.uint16)
    else:
        tile = tile.astype(np.uint16)
    return tile

def process_single_slice(slice_img, slice_idx, grid, scale_12bit, image_name, crop_coords):
    """
    For a given 2D slice, first apply filtering, then crop using fixed coordinates,
    then tile the cropped region into grid patches.
    Returns a list of delayed tile tasks.
    """
    # Apply filtering (unsharp mask, top-hat, black-hat)
    enhanced_slice = apply_filters(slice_img)
    
    # Get fixed crop coordinates for this acquisition.
    # crop_coords values: (crop_top, crop_left, crop_bottom, crop_right)
    (crop_top, crop_left, crop_bottom, crop_right) = crop_coords[image_name]
    cropped_img = enhanced_slice[crop_top:crop_bottom, crop_left:crop_right]
    crop_height = crop_bottom - crop_top
    crop_width = crop_right - crop_left
    
    grid_rows, grid_cols = grid
    tile_h = crop_height // grid_rows
    tile_w = crop_width // grid_cols

    tasks = []
    for i in range(grid_rows):
        for j in range(grid_cols):
            row_start = i * tile_h
            row_end = (i + 1) * tile_h
            col_start = j * tile_w
            col_end = (j + 1) * tile_w
            tile = cropped_img[row_start:row_end, col_start:col_end]
            tasks.append(process_tile(tile, scale_12bit))
    return tasks

def crop_and_tile_zarr_stack(zarr_path, grid=(3,3), output_dir="tiles", scale_12bit=False):
    """
    For the given Zarr file, process each 2D slice as follows:
      - For acquisition 5, limit to the first 90 slices.
      - For each slice, apply filtering, then crop using fixed coordinates,
        and tile into grid patches.
    Then, group the resulting tiles by grid position (total 9 groups for grid=(3,3))
    and save each group as a multipage TIFF stack in a folder for that acquisition.
    """
    img = get_image_from_zarr_as_dask_array(zarr_path)
    image_name = os.path.splitext(os.path.basename(zarr_path))[0]
    
    # Fixed crop coordinates for each acquisition.
    crop_coords = {
        "MUSE_stitched_acq_3": (3050, 1612, 4959, 5413),
        "MUSE_stitched_acq_4": (3047, 1416, 5089, 5052),
        "MUSE_stitched_acq_5": (2935, 1394, 4953, 5126)
    }
    
    # Create a folder for this acquisition.
    acq_folder = os.path.join(output_dir, image_name)
    if not os.path.exists(acq_folder):
        os.makedirs(acq_folder)
    
    num_slices = img.shape[0] if img.ndim == 3 else 1
    if image_name == "MUSE_stitched_acq_5":
        num_slices = min(num_slices, 76)
    
    all_slice_tiles = []  # List over slices; each entry is a list of tiles (order: grid positions)
    for slice_idx in range(num_slices):
        slice_img = img[slice_idx] if img.ndim == 3 else img
        slice_tasks = process_single_slice(slice_img, slice_idx, grid, scale_12bit, image_name, crop_coords)
        slice_tiles = compute(*slice_tasks, scheduler='synchronous')
        all_slice_tiles.append(slice_tiles)
    
    # Group tiles by grid position.
    num_tile_positions = grid[0] * grid[1]
    tile_groups = [[] for _ in range(num_tile_positions)]
    for slice_tiles in all_slice_tiles:
        for idx, tile in enumerate(slice_tiles):
            if tile is not None:
                tile_groups[idx].append(tile)
    
    # Save each group as a multipage TIFF stack.
    for idx, group in enumerate(tile_groups):
        if group:
            stack = np.stack(group, axis=0)
            out_file = os.path.join(acq_folder, f"{image_name}_tile_{idx:03}_stack.tiff")
            tif.imwrite(out_file, stack)
        else:
            print(f"No tiles for group {idx} in {image_name}")

def process_selected_zarrs(input_dir, output_dir, selected_acquisitions, grid=(3,3), scale_12bit=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for acq in tqdm(selected_acquisitions, desc="Processing Zarr images"):
        filename = f"MUSE_stitched_acq_{acq}.zarr"
        zarr_path = os.path.join(input_dir, filename)
        try:
            crop_and_tile_zarr_stack(zarr_path, grid=grid, output_dir=output_dir, scale_12bit=scale_12bit)
        except Exception as e:
            print(f"Error processing {zarr_path}: {e}")

# --- Run the Process ---
input_dir = "/Volumes/T9/Virtual_Histology/source_data/LFB-Rhod-48Fix-DMSO/"
output_dir = "/Volumes/T9/Virtual_Histology/primary_data/processed_zarr_tiles"
grid = (5,5)  # This will yield 9 tiles per slice.
selected_acquisitions = [3, 4, 5]
scale_12bit = True

dask.config.set({'temporary_directory': '/tmp/dask'})

process_selected_zarrs(input_dir, output_dir, selected_acquisitions, grid=grid, scale_12bit=scale_12bit)

print("Processing complete.")


Processing Zarr images: 100%|██████████| 3/3 [07:25<00:00, 148.46s/it]

Processing complete.


In [2]:
import os
import numpy as np
import tifffile as tiff
import dask.array as da
from dask import delayed, compute
from tqdm import tqdm
import dask

# skimage stuff
from skimage.filters import unsharp_mask
from skimage.util import img_as_float

############################
# 1) Zarr -> Dask Reading
############################
def get_image_from_zarr_as_dask_array(zarr_path):
    zimg = da.from_zarr(zarr_path, component="muse/stitched/")
    return zimg

############################
# 2) Filtering
############################
def apply_filters(img):
    """
    Convert to float, do unsharp_mask, etc. Return float in [0,1].
    """
    img_float = img_as_float(img)
    sharp = unsharp_mask(img_float, radius=1, amount=3)
    return sharp

############################
# 3) Delayed Processing
############################
@delayed
def process_tile(tile, scale_12bit):
    """
    Convert tile to np.uint16, optionally scale from 12-bit to 16-bit range.
    """
    tile = tile.compute(scheduler='synchronous') if hasattr(tile, 'compute') else tile
    if tile.size == 0:
        return None

    tile = tile.astype(np.float32)
    tile_max = tile.max()
    if tile_max <= 0:
        tile_final = np.zeros_like(tile, dtype=np.uint16)
    else:
        if scale_12bit:
            # check if ~12-bit
            if abs(tile_max - 4095) / 4095 < 0.1:
                scale_factor = 16.0
            else:
                scale_factor = 65535.0 / tile_max
            tile_final = np.clip(tile*scale_factor,0,65535).astype(np.uint16)
        else:
            # just cast to uint16
            tile_final = np.clip(tile*65535,0,65535).astype(np.uint16)
    return tile_final

############################
# 4) Overlapping Single Slice
############################
def process_single_slice_overlap(
    slice_img,
    slice_idx,
    tile_size,         # (tile_h,tile_w)
    overlap,           # (ov_h,ov_w)
    scale_12bit,
    image_name,
    crop_coords
):
    """
    1) Filter the slice,
    2) Crop using crop_coords,
    3) Overlap tile with skipping partial edges => all tiles same shape => np.stack
    """
    # A) Filter
    enhanced_slice = apply_filters(slice_img)

    # B) Crop
    (crop_top, crop_left, crop_bottom, crop_right) = crop_coords[image_name]
    cropped_img = enhanced_slice[crop_top:crop_bottom, crop_left:crop_right]
    crop_height = crop_bottom - crop_top
    crop_width  = crop_right - crop_left

    tile_h, tile_w = tile_size
    ov_h, ov_w = overlap

    tasks = []
    # stride
    stride_h = tile_h - ov_h
    stride_w = tile_w - ov_w

    row = 0
    while row + tile_h <= crop_height:  # skip partial bottom
        col = 0
        while col + tile_w <= crop_width:  # skip partial right
            tile = cropped_img[row:row+tile_h, col:col+tile_w]
            tasks.append(process_tile(tile, scale_12bit))
            col += stride_w
        row += stride_h

    return tasks

############################
# 5) Main Crop & Tile Zarr
############################
def crop_and_tile_zarr_stack(
    zarr_path,
    tile_size=(512,512),
    overlap=(64,64),
    output_dir="tiles",
    scale_12bit=False
):
    """
    For each 2D slice in the Zarr stack:
      - Filter
      - Crop with fixed coords
      - Overlap tile (skip partial edges)
    We then group all tiles from each slice, stack them => one TIF per slice
    (like "MUSE_stitched_acq_4_slice_000_tiles.tiff").
    """

    img = get_image_from_zarr_as_dask_array(zarr_path)
    image_name = os.path.splitext(os.path.basename(zarr_path))[0]

    # crop coords per acquisition
    crop_coords = {
        "MUSE_stitched_acq_4": (2530, 1569, 5244, 5215),
        # Add others as needed
    }

    acq_folder = os.path.join(output_dir, image_name)
    os.makedirs(acq_folder, exist_ok=True)

    num_slices = img.shape[0] if img.ndim==3 else 1
    if image_name=="MUSE_stitched_acq_5":
        num_slices = min(num_slices, 76)

    all_slice_tiles = []
    for slice_idx in range(num_slices):
        slice_img = img[slice_idx] if img.ndim==3 else img
        slice_tasks = process_single_slice_overlap(
            slice_img,
            slice_idx,
            tile_size,
            overlap,
            scale_12bit,
            image_name,
            crop_coords
        )
        slice_tiles = compute(*slice_tasks, scheduler='synchronous')
        # remove None
        slice_tiles = [t for t in slice_tiles if t is not None]
        all_slice_tiles.append(slice_tiles)

    # Now we store each slice's tiles in a multipage TIF => same shape => (tile_h,tile_w)
    for slice_idx, tile_list in enumerate(all_slice_tiles):
        if not tile_list:
            continue
        # All tiles have shape => (tile_h,tile_w), so we can do np.stack
        stack = np.stack(tile_list, axis=0)  # (N, tile_h, tile_w)
        out_file = os.path.join(acq_folder, f"{image_name}_slice_{slice_idx:03}_tiles.tiff")
        tiff.imwrite(out_file, stack)
        print(f"Wrote {stack.shape} => {out_file}")

############################
# 6) Process multiple acquisitions
############################
def process_selected_zarrs(
    input_dir,
    output_dir,
    selected_acquisitions,
    tile_size=(512,512),
    overlap=(64,64),
    scale_12bit=False
):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    from tqdm import tqdm
    for acq in tqdm(selected_acquisitions, desc="Processing Zarr images"):
        filename = f"MUSE_stitched_acq_{acq}.zarr"
        zarr_path = os.path.join(input_dir, filename)
        try:
            crop_and_tile_zarr_stack(
                zarr_path,
                tile_size=tile_size,
                overlap=overlap,
                output_dir=output_dir,
                scale_12bit=scale_12bit
            )
        except Exception as e:
            print(f"Error processing {zarr_path}: {e}")

############################
# 7) Example "Main"
############################
if __name__=="__main__":
    input_dir  = "/Volumes/T9/Virtual_Histology/source_data/LFB-Rhod-48Fix-DMSO/"
    output_dir = "/Volumes/T9/Virtual_Histology/primary_data/"
    tile_size  = (512,512)
    overlap    = (64,64)
    selected_acquisitions = [3,4,5]
    scale_12bit = True
    dask.config.set({'temporary_directory': '/tmp/dask'})
    process_selected_zarrs(
        input_dir,
        output_dir,
        selected_acquisitions,
        tile_size=tile_size,
        overlap=overlap,
        scale_12bit=scale_12bit
    )
    print("Processing complete. Overlapping tiles (no partial edges).")


Processing Zarr images:  33%|███▎      | 1/3 [00:01<00:02,  1.11s/it]

Error processing /Volumes/T9/Virtual_Histology/source_data/LFB-Rhod-48Fix-DMSO/MUSE_stitched_acq_3.zarr: 'MUSE_stitched_acq_3'
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_000_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_001_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_002_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_003_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_004_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_005_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_

Processing Zarr images:  67%|██████▋   | 2/3 [04:10<02:27, 147.03s/it]

Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_248_tiles.tiff
Wrote (35, 512, 512) => /Volumes/T9/Virtual_Histology/primary_data/MUSE_stitched_acq_4/MUSE_stitched_acq_4_slice_249_tiles.tiff


Processing Zarr images: 100%|██████████| 3/3 [04:11<00:00, 83.79s/it] 

Error processing /Volumes/T9/Virtual_Histology/source_data/LFB-Rhod-48Fix-DMSO/MUSE_stitched_acq_5.zarr: 'MUSE_stitched_acq_5'
Processing complete. Overlapping tiles (no partial edges).
